<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 3.5: Object Oriented Programming
**Prev: [Functional Programming](3.4_functional_programming.ipynb)**<br>
**Next: [Types](3.6_types.ipynb)**

## Motivation
Scala와 Chisel은 객체 지향 프로그래밍 언어입니다. 즉, 코드를 객체로 구분할 수 있습니다. Java를 기반으로 하는 Scala는 Java의 많은 객체 지향 기능을 상속합니다. 그러나 아래에서 볼 수 있듯이 몇 가지 차이점이 있습니다. Chisel의 하드웨어 모듈은 단일 또는 다중 인스턴스로 인스턴스화 및 연결될 수 있다는 점에서 Verilog의 모듈과 유사합니다.

## Setup

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.experimental._
import chisel3.tester._
import chisel3.tester.RawTester.test

---
# Object Oriented Programming
이 섹션에서는 Scala가 객체 지향 프로그래밍 패러다임을 구현하는 방법을 설명합니다. 지금까지 클래스를 이미 보았지만 Scala에는 다음과 같은 기능도 있습니다.
- [Abstract classes](#abstract)
- [Traits](#traits)
- [Objects](#objects)
- [Companion Objects](#compobj)
- [Case Classes](#caseclass)

## Abstract Classes<a name="abstract"></a>
추상 클래스는 다른 프로그래밍 언어 구현과 같습니다. 하위 클래스가 구현해야 하는 많은 구현되지 않은 값을 정의할 수 있습니다. 모든 개체는 하나의 상위 추상 클래스에서만 직접 상속할 수 있습니다.

<span style="color:blue">**Example: Abstract Class**</span><br>

In [2]:
abstract class MyAbstractClass {
  def myFunction(i: Int): Int
  val myValue: String
}
class ConcreteClass extends MyAbstractClass {
  def myFunction(i: Int): Int = i + 1
  val myValue = "Hello World!"
}
// Uncomment below to test!
// val abstractClass = new MyAbstractClass() // Illegal! Cannot instantiate an abstract class
val concreteClass = new ConcreteClass()      // Legal!


defined class MyAbstractClass
defined class ConcreteClass
concreteClass: ConcreteClass = ammonite.$sess.cmd1$Helper$ConcreteClass@189b1524

## Traits<a name="traits"></a>
특성은 구현되지 않은 값을 정의할 수 있다는 점에서 추상 클래스와 매우 유사합니다. 그러나 두 가지 면에서 다릅니다.
- 클래스는 여러 특성을 상속할 수 있습니다.
- 트레잇은 생성자 매개변수를 가질 수 없습니다.

<span style="color:blue">**Example: Traits and Multiple Inheritance**</span><br>
특성은 아래 예제와 같이 Scala가 다중 상속을 구현하는 방법입니다. `MyClass`는 `HasFunction` 및 `HasValue` 특성에서 확장됩니다.

In [ ]:
trait HasFunction {
  def myFunction(i: Int): Int
}
trait HasValue {
  val myValue: String
  val myOtherValue = 100
}
class MyClass extends HasFunction with HasValue {
  override def myFunction(i: Int): Int = i + 1
  val myValue = "Hello World!"
}
// Uncomment below to test!
// val myTraitFunction = new HasFunction() // Illegal! Cannot instantiate a trait
// val myTraitValue = new HasValue()       // Illegal! Cannot instantiate a trait
val myClass = new MyClass()                // Legal!

여러 특성을 상속하려면 다음과 같이 연결하십시오.

```scala
class MyClass extends HasTrait1 with HasTrait2 with HasTrait3 ...
```
일반적으로 추상 클래스의 단일 상속 제한을 적용하려는 것이 확실하지 않은 한 항상 추상 클래스보다 특성을 사용하십시오.

## Objects<a name="objects"></a>
Scala에는 객체라고 하는 이러한 싱글톤 클래스에 대한 언어 기능이 있습니다. 객체를 인스턴스화할 수 없습니다 **(`new`를 호출할 필요 없음)**; 단순히 직접 참조할 수 있습니다. 따라서 Java 정적 클래스와 유사합니다.

<span style="color:blue">**Example: Objects**</span><br>

In [ ]:
object MyObject {
  def hi: String = "Hello World!"
  def apply(msg: String) = msg
}
println(MyObject.hi)
println(MyObject("This message is important!")) // equivalent to MyObject.apply(msg)

## Companion Objects<a name="compobj"></a>

클래스와 객체가 동일한 이름을 공유하고 동일한 파일에 정의된 경우 객체를 **컴패니언 객체**라고 합니다. 클래스/객체 이름 앞에 'new'를 사용하면 클래스가 인스턴스화됩니다. `new`를 사용하지 않으면 객체를 참조합니다.

<span style="color:blue">**Example: Companion Object**</span><br>

In [ ]:
object Lion {
    def roar(): Unit = println("I'M AN OBJECT!")
}
class Lion {
    def roar(): Unit = println("I'M A CLASS!")
}
new Lion().roar()
Lion.roar()

컴패니언 개체는 일반적으로 다음과 같은 이유로 사용됩니다.
   1. 클래스와 관련된 상수를 포함하기 위해
   2. 클래스 생성자 앞/뒤에 코드 실행
   3. 클래스에 대해 여러 생성자를 생성하기 위해

아래 예에서는 Animal의 여러 인스턴스를 인스턴스화합니다. 우리는 각 동물이 이름을 갖고 모든 인스턴스 내에서 그 순서를 알기를 원합니다. 마지막으로 이름이 지정되지 않으면 기본 이름을 가져와야 합니다.

In [ ]:
object Animal {
    val defaultName = "Bigfoot"
    private var numberOfAnimals = 0
    def apply(name: String): Animal = {
        numberOfAnimals += 1
        new Animal(name, numberOfAnimals)
    }
    def apply(): Animal = apply(defaultName)
}
class Animal(name: String, order: Int) {
  def info: String = s"Hi my name is $name, and I'm $order in line!"
}

val bunny = Animal.apply("Hopper") // Calls the Animal factory method
println(bunny.info)
val cat = Animal("Whiskers")       // Calls the Animal factory method
println(cat.info)
val yeti = Animal()                // Calls the Animal factory method
println(yeti.info)


*What's happening here?*
1. **Animal Companion Object**는 ```class Animal```과 관련된 상수를 정의합니다.:
```scala
val defaultName = "Bigfoot"
```
1. 또한 Animal 인스턴스의 순서를 추적하기 위해 개인용 가변 정수를 정의합니다.:
```scala 
private var numberOfAnimals = 0
```
1. 그것은 **class Animal**의 인스턴스를 반환한다는 점에서 **factory 메소드**로 알려진 두 개의 **apply** 메소드를 정의합니다. 
    1. 첫 번째는 하나의 인수 ```name```만 사용하여 Animal 인스턴스를 만들고 ```numberOfAnimals```를 사용하여 Animal 클래스 생성자를 호출합니다.
```scala
def apply(name: String): Animal = {
            numberOfAnimals += 1
            new Animal(name, numberOfAnimals)
}
```
    1. 두 번째 팩토리 메소드는 인수가 필요하지 않으며 대신 기본 이름을 사용하여 다른 적용 메소드를 호출합니다.
```scala
def apply(): Animal = apply(defaultName)
```
1. 이러한 팩토리 메서드는 다음과 같이 순진하게 호출할 수 있습니다.
```scala
val bunny = Animal.apply("Hopper")
```
이렇게 하면 new 키워드를 사용할 필요가 없지만 실제 마법은 컴파일러가 인스턴스 또는 개체에 적용된 괄호를 볼 때마다 적용 메서드를 가정한다는 것입니다.
```scala
val cat = Animal("Whiskers")
```
1. 일반적으로 컴패니언 개체를 통해 제공되는 팩토리 메서드를 사용하면 인스턴스 생성을 표현하고 생성자 매개변수, 변환에 대한 추가 테스트를 제공하고 키워드 ```new```를 사용할 필요가 없습니다. 'numberOfAnimals'가 증가하려면 컴패니언 개체의 'apply' 메서드를 호출해야 합니다.

다음을 작성할 때 **Chisel은 Module과 같은 많은 도우미 개체를 사용합니다.**:
```scala
val myModule = Module(new MyModule)
```
**모듈 컴패니언 개체**를 호출하므로 Chisel은 인스턴스화 전후에 백그라운드 코드를 실행할 수 있습니다.
```MyModule```.

## Case Classes<a name="caseclass"/>
케이스 클래스는 멋진 추가 기능을 제공하는 특수한 유형의 Scala 클래스입니다. 스칼라 프로그래밍에서 매우 일반적이므로 이 섹션에서는 몇 가지 유용한 기능에 대해 간략히 설명합니다.
- **class parameters**에 대한 **external access** 허용
- 클래스를 인스턴스화할 때 **`new`**를 사용할 필요를 **제거**
- 모든 클래스 매개변수에 대한 액세스를 제공하는 **unapply method**를 자동으로 생성합니다.
- 하위 분류할 수 없음

다음 예제에서는 `Nail`, `Screw`, `Staple`의 세 가지 다른 클래스를 선언합니다.

In [ ]:
class Nail(length: Int) // Regular class
val nail = new Nail(10) // Requires the `new` keyword
// println(nail.length) // Illegal! Class constructor parameters are not by default externally visible

class Screw(val threadSpace: Int) // By using the `val` keyword, threadSpace is now externally visible
val screw = new Screw(2)          // Requires the `new` keyword
println(screw.threadSpace)

case class Staple(isClosed: Boolean) // Case class constructor parameters are, by default, externally visible
val staple = Staple(false)           // No `new` keyword required
println(staple.isClosed)

`Nail`은 일반 클래스이며 인수 목록에서 `val` 키워드를 사용하지 않았기 때문에 매개변수는 외부에서 볼 수 없습니다. 또한 `Nail`의 인스턴스를 선언할 때 `new` 키워드가 필요합니다.

`Screw`는 `Nail`과 유사하게 선언되지만 인수 목록에 `val`이 포함됩니다. 이렇게 하면 매개변수 `threadSpace`를 외부에서 볼 수 있습니다.

케이스 클래스를 사용하여 `Staple`은 모든 매개변수를 외부에서 볼 수 있다는 이점을 얻습니다(`val` 키워드가 필요 없음).

또한 `Staple`은 케이스 클래스를 선언할 때 `new`를 사용할 필요가 없습니다. 이는 Scala 컴파일러가 케이스 클래스에 대한 적용 메소드를 포함하는 코드의 모든 케이스 클래스에 대한 컴패니언 객체를 자동으로 생성하기 때문입니다.

케이스 클래스는 매개변수가 많은 생성기를 위한 좋은 컨테이너입니다.
생성자는 파생 매개변수를 정의하고 입력의 유효성을 검사하기에 좋은 위치를 제공합니다.

In [3]:
case class SomeGeneratorParameters(
    someWidth: Int,
    someOtherWidth: Int = 10,
    pipelineMe: Boolean = false
) {
    require(someWidth >= 0)
    require(someOtherWidth >= 0)
    val totalWidth = someWidth + someOtherWidth
}

defined class SomeGeneratorParameters

---
# Inheritance with Chisel<a name="super"></a>
이전에 `Module`과 `Bundle`을 보았지만 실제로 무슨 일이 일어나고 있는지 깨닫는 것이 중요합니다. 여러분이 만드는 모든 Chisel 모듈은 기본 유형인 `Module`을 확장하는 클래스입니다. 만드는 모든 Chisel IO는 기본 유형 `Bundle`(또는 일부 특별한 경우에는 `Bundle`의 상위 유형 [`Record`](https://github.com/freechipsproject/chisel3/blob/v3.0.0/chiselFrontend/src/main/scala/chisel3/core/Aggregate.scala#L415))을 확장하는 클래스입니다. `UInt` 또는 `Bundle`과 같은 치즐 하드웨어 유형에는 모두 `Data`가 상위 유형으로 있습니다. 계층적 하드웨어 블록을 생성하고 객체 재사용을 탐색하기 위해 객체 지향 프로그래밍을 사용하는 방법을 살펴보겠습니다. 유형 제네릭 생성기에 대한 다음 모듈에서 유형과 `데이터`에 대해 자세히 알아볼 것입니다.



## Module<a name="module"></a>
Chisel에서 하드웨어 개체를 생성하려면 'Module'이 슈퍼클래스로 있어야 합니다.
상속이 재사용을 위한 올바른 도구가 아닐 수도 있지만([상속보다 합성](https://en.wikipedia.org/wiki/Composition_over_inheritance)은 일반적인 원칙입니다), 상속은 여전히 강력한 도구입니다.
아래는 `Module`을 만들고 여러 인스턴스를 계층적으로 연결하는 예입니다.

<span style="color:blue">**Example: Gray Encoder and Decoder**</span><br>
하드웨어 Gray 인코더/디코더를 생성하겠습니다. 인코딩 또는 디코딩 작업 선택은 하드웨어 프로그래밍이 가능합니다.

In [3]:
import scala.math.pow

// create a module
class GrayCoder(bitwidth: Int) extends Module {
  val io = IO(new Bundle{
    val in = Input(UInt(bitwidth.W))
    val out = Output(UInt(bitwidth.W))
    val encode = Input(Bool()) // decode on false
  })
  
  when (io.encode) { //encode
    io.out := io.in ^ (io.in >> 1.U)
  } .otherwise { // decode, much more complicated
    io.out := Seq.fill(log2Ceil(bitwidth))(Wire(UInt(bitwidth.W))).zipWithIndex.fold((io.in, 0)){
      case ((w1: UInt, i1: Int), (w2: UInt, i2: Int)) => {
        w2 := w1 ^ (w1 >> pow(2, log2Ceil(bitwidth)-i2-1).toInt.U)
        (w2, i1)
      }
    }._1
  }
}


cmd3.sc:4: not found: type Module
class GrayCoder(bitwidth: Int) extends Module {
                                       ^cmd3.sc:5: not found: value IO
  val io = IO(new Bundle{
           ^cmd3.sc:5: not found: type Bundle
  val io = IO(new Bundle{
                  ^cmd3.sc:6: not found: value UInt
    val in = Input(UInt(bitwidth.W))
                   ^cmd3.sc:6: value W is not a member of Int
    val in = Input(UInt(bitwidth.W))
                                 ^cmd3.sc:7: not found: value Output
    val out = Output(UInt(bitwidth.W))
              ^cmd3.sc:7: not found: value UInt
    val out = Output(UInt(bitwidth.W))
                     ^cmd3.sc:7: value W is not a member of Int
    val out = Output(UInt(bitwidth.W))
                                   ^cmd3.sc:8: not found: value Bool
    val encode = Input(Bool()) // decode on false
                       ^cmd3.sc:11: not found: value when
  when (io.encode) { //encode
  ^Compilation Failed

: 

Give it a whirl!

In [3]:
// test our gray coder
val bitwidth = 4
test(new GrayCoder(bitwidth)) { c =>
    def toBinary(i: Int, digits: Int = 8) = {
        String.format("%" + digits + "s", i.toBinaryString).replace(' ', '0')
    }
    println("Encoding:")
    for (i <- 0 until pow(2, bitwidth).toInt) {
        c.io.in.poke(i.U)
        c.io.encode.poke(true.B)
        c.clock.step(1)
        println(s"In = ${toBinary(i, bitwidth)}, Out = ${toBinary(c.io.out.peek().litValue.toInt, bitwidth)}")
    }

    println("Decoding:")
    for (i <- 0 until pow(2, bitwidth).toInt) {
        c.io.in.poke(i.U)
        c.io.encode.poke(false.B)
        c.clock.step(1)
        println(s"In = ${toBinary(i, bitwidth)}, Out = ${toBinary(c.io.out.peek().litValue.toInt, bitwidth)}")
    }

}

cmd3.sc:2: not found: value test
val res3_1 = test(new GrayCoder(bitwidth)) { c =>
             ^cmd3.sc:2: not found: type GrayCoder
val res3_1 = test(new GrayCoder(bitwidth)) { c =>
                      ^cmd3.sc:7: not found: value pow
    for (i <- 0 until pow(2, bitwidth).toInt) {
                      ^cmd3.sc:15: not found: value pow
    for (i <- 0 until pow(2, bitwidth).toInt) {
                      ^Compilation Failed

: 

그레이 코드는 종종 비동기 인터페이스에서 사용됩니다. 일반적으로 완전한 기능을 갖춘 인코더/디코더보다 회색 카운터가 사용되지만 위의 모듈을 사용하여 작업을 단순화합니다. 아래는 위의 Gray 코더를 사용하여 빌드된 AsyncFIFO의 예입니다. 제어 로직과 테스터는 나중에 연습으로 남겨둡니다. 지금은 Gray 코더가 여러 번 인스턴스화되고 연결되는 방법을 살펴보십시오.

In [3]:
class AsyncFIFO(depth: Int = 16) extends Module {
  val io = IO(new Bundle{
    // write inputs
    val write_clock = Input(Clock())
    val write_enable = Input(Bool())
    val write_data = Input(UInt(32.W))
    
    // read inputs/outputs
    val read_clock = Input(Clock())
    val read_enable = Input(Bool())
    val read_data = Output(UInt(32.W))
    
    // FIFO status
    val full = Output(Bool())
    val empty = Output(Bool())
  })
  
  // add extra bit to counter to check for fully/empty status
  assert(isPow2(depth), "AsyncFIFO needs a power-of-two depth!")
  val write_counter = withClock(io.write_clock) { Counter(io.write_enable && !io.full, depth*2)._1 }
  val read_counter = withClock(io.read_clock) { Counter(io.read_enable && !io.empty, depth*2)._1 }
  
  // encode
  val encoder = new GrayCoder(write_counter.getWidth)
  encoder.io.in := write_counter
  encoder.io.encode := true.B
  
  // synchronize
  val sync = withClock(io.read_clock) { ShiftRegister(encoder.io.out, 2) }
  
  // decode
  val decoder = new GrayCoder(read_counter.getWidth)
  decoder.io.in := sync
  decoder.io.encode := false.B
  
  // status logic goes here
  
}

cmd3.sc:1: not found: type Module
class AsyncFIFO(depth: Int = 16) extends Module {
                                         ^cmd3.sc:2: not found: value IO
  val io = IO(new Bundle{
           ^cmd3.sc:2: not found: type Bundle
  val io = IO(new Bundle{
                  ^cmd3.sc:4: not found: value Clock
    val write_clock = Input(Clock())
                            ^cmd3.sc:5: not found: value Bool
    val write_enable = Input(Bool())
                             ^cmd3.sc:6: not found: value UInt
    val write_data = Input(UInt(32.W))
                           ^cmd3.sc:6: value W is not a member of Int
    val write_data = Input(UInt(32.W))
                                   ^cmd3.sc:9: not found: value Clock
    val read_clock = Input(Clock())
                           ^cmd3.sc:10: not found: value Bool
    val read_enable = Input(Bool())
                            ^cmd3.sc:11: not found: value Output
    val read_data = Output(UInt(32.W))
                    ^cmd3.sc:11: not 

: 

---
# You're done!

[Return to the top.](#top)